<a href="https://colab.research.google.com/github/pvbarve29/Amazon-Reviews-Data/blob/master/lstm1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import bz2
import re
from matplotlib import pyplot as plt
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding,LSTM,Dropout,Dense
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from keras.layers import SpatialDropout1D

#trainFile='/content/drive/My Drive/train.ft.txt.bz2'
#file=bz2.BZ2File(trainFile,'r')
#lines=file.readlines()

train_file = bz2.BZ2File('/content/drive/My Drive/BE Project/Sentiment analysis/datasets/train.ft.txt.bz2')
train_lines = train_file.readlines()
train_lines = [x.decode('utf-8') for x in train_lines]

test_file = bz2.BZ2File('/content/drive/My Drive/BE Project/Sentiment analysis/datasets/test.ft.txt.bz2')
test_lines = test_file.readlines()
test_lines = [x.decode('utf-8') for x in test_lines]

docSentimentList=[]
def getDocumentSentimentList(docs,splitStr='__label__'):
    for i in range(len(docs)):
        text=str(docs[i])
        splitText=text.split(splitStr)
        secHalf=splitText[1]
        text=secHalf[2:len(secHalf)-1]
        sentiment=secHalf[0]
        docSentimentList.append([text,sentiment])
    return docSentimentList
  
docSentimentList=getDocumentSentimentList(train_lines,splitStr = '__label__')
train_DF=pd.DataFrame(docSentimentList,columns=['TEXT','SENTIMENT'])
X_train=train_DF['TEXT']
y_train=train_DF['SENTIMENT']
y_train=y_train.astype('int32')
lb=LabelBinarizer(pos_label=1,neg_label=0)
y_train_binarized=lb.fit_transform(y_train)
y_train=to_categorical(num_classes=2,y=y_train_binarized)


docSentimentList=getDocumentSentimentList(test_lines,splitStr = '__label__')
test_DF=pd.DataFrame(docSentimentList,columns=['TEXT','SENTIMENT'])
X_test=test_DF['TEXT']
y_test=test_DF['SENTIMENT']
y_test=y_test.astype('int32')
lb=LabelBinarizer(pos_label=1,neg_label=0)
y_test_binarized=lb.fit_transform(y_test)
y_test=to_categorical(num_classes=2,y=y_test_binarized)

print("tokenizing training data")
tok=Tokenizer(num_words=100000,lower=True)
tok.fit_on_texts(X_train)
print("Sequencing training data")
seqs=tok.texts_to_sequences(X_train)
train_padded_seqs=pad_sequences(seqs,maxlen=100)

print("tokenizing testing data")
tok.fit_on_texts(X_test)
print("tokenizing done!!")
print("Sequencing testing data")
seqs1=tok.texts_to_sequences(X_test)
test_padded_seqs=pad_sequences(seqs1,maxlen=100)
print("Sequencing done!!!")

def createLSTM():
    model = Sequential()
    model.add(Embedding(100000, 100))
    model.add(SpatialDropout1D(0.4))
    model.add(LSTM(512, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(2,activation='softmax'))
    return model
model=createLSTM()

model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
earlyStopping=EarlyStopping(patience=3,monitor='acc',min_delta=0.0001,verbose=1)
callbackslist=[earlyStopping]
model.fit(X_train,y_train,batch_size=512,epochs=5,verbose=1,callbacks=callbackslist,validation_split = 0.1)

loss,acc = model.evaluate(X_test, y_test, verbose = 1, batch_size = 512)
print("loss = "+ str(loss) + "accuracy = "+str(acc))

idx=np.random.randint(len(X))
test=[X[idx]]
print(test)
print('RESULT:')
pred=model.predict(pad_sequences(tok.texts_to_sequences(test),maxlen=100))
print(np.argmax(pred))
if np.argmax(pred)==0:
    print('NEG')
else:
    print('POS')

Using TensorFlow backend.


tokenizing training data
Sequencing training data
tokenizing testing data
tokenizing done!!
Sequencing testing data
